In [6]:
import pandas as pd
#import requests
import json
#import yaml
import time
import glob

import datetime as dt
from collections import Counter
import matplotlib.pyplot as plt

import numpy as np

from ipywidgets import interactive, interact
from IPython.display import Image
import ipywidgets as widgets

from myTwitterLibrary import imageScroll
from myTwitterLibrary import Twitter_analysis as mta
from myTwitterLibrary import myTwitterLibrary as mtl
from myTwitterLibrary import nlp
from myTwitterLibrary import analysis as analysis


pd.set_option('display.max_colwidth', None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
df=pd.read_json("2023-05-05ChatGpt_all.json")


# Start the exploration with a wordcloud or wordcount :)

In [8]:
@interact(column=["hashtags","NoStopwords"],Mostcommon=(5,200))
def make_wordcloud(column="NoStopwords",Mostcommon=20,Save_as=""):
    hashtags=[i.lower() for s in df[column].dropna() for i in s] # hashtags OR # no stopwords
    analysis.make_wordcloud(hashtags,filename=Save_as,Mostcommon=Mostcommon)

interactive(children=(Dropdown(description='column', index=1, options=('hashtags', 'NoStopwords'), value='NoSt…

In [10]:
@widgets.interact(column=["NoStopwords","hashtags","mentions","urls"],n=(1,50))
def mostCommon(column="NoStopwords",n=10):
    analysis.mostCommon(df,column=column,n=n)
    

interactive(children=(Dropdown(description='column', options=('NoStopwords', 'hashtags', 'mentions', 'urls'), …

# Filter & discover popular Tweets

In [11]:
@interact(n_tweets=(1,50),start="2012-11-01",end="2023-03-30",like_count=(0,100),replay_count=(0,100),retweet_count=(0,100),word_count=(1,df.word_count.max()), polarity=(-1,1,0.2),language=df.language.value_counts()[:5].index)
def filter_df(save=False,n_tweets=5,start="2012-11-01",end="2023-03-30",like_count=30,replay_count=0,retweet_count=0,word_count=5,PolarityFilter=False,polarity=0,language="no filter",Wordfilter="", Userfilter=""):
    analysis.filter_df(df,save,n_tweets,start,end,like_count,replay_count,retweet_count,word_count,polarity,language,Wordfilter=Wordfilter,Userfilter=Userfilter, PolarityFilter=PolarityFilter)
  

interactive(children=(Checkbox(value=False, description='save'), IntSlider(value=5, description='n_tweets', ma…

# Counting Users & Elements.

In [12]:
@interact(by_value=['like_count',"retweet_count","reply_count","quote_count","impression_count","word_count","letters_count","polarity"],n=(3,100),aggFunk=["mean","sum"])
def most_popular_users(by_value='like_count',n=10,aggFunk="sum"):
    analysis.most_popular_users(df, by_value=by_value,n=n,plot=True,DataName=False,aggFunk=aggFunk)



interactive(children=(Dropdown(description='by_value', options=('like_count', 'retweet_count', 'reply_count', …

In [13]:
@interact(column=["hashtags","NoStopwords","language","urls"],n=(3,100))
def mostCommon(column="NoStopwords",n=5): 
    analysis.mostCommon(df,column,n) 

interactive(children=(Dropdown(description='column', index=1, options=('hashtags', 'NoStopwords', 'language', …

# Words in Context

In [14]:
@interact(column=["hashtags","NoStopwords"],Nodes=(3,300),minDegree=(1,10),removeIsolates=True,only_largest_component=True)
def NetworkAnalysis(column="hashtags",Nodes=20,minDegree=5,removeIsolates=True,only_largest_component=True):
    G=analysis.Word_NetworkGen(df,n=Nodes,column=column)
    G=analysis.CleanGraph(G,removeIsolates=removeIsolates,minDegree=minDegree,only_largest_component=only_largest_component)
    analysis.writeNetworkHTML(G,view=False)
print("see the network: The_network.html")

interactive(children=(Dropdown(description='column', options=('hashtags', 'NoStopwords'), value='hashtags'), I…

see the network: The_network.html


In [15]:
#This cell allows to read the words before and after a search_word. You can change the searchword & the size of the context.
@interact(search_word="sustainable",words_before=(0,10),words_after=(0,10))
def text_context(column=["text","pure_text","NoStopwords"],search_word="sustainable",words_before=1,words_after=1):
    df[column].dropna().apply(analysis.Keyword_context,search_word=search_word.lower(),context=(words_before,words_after))

    


interactive(children=(Dropdown(description='column', options=('text', 'pure_text', 'NoStopwords'), value='text…

# Image Analysis

In [17]:
from myTwitterLibrary import imageScroll
imgs=df[df.media_url.apply(lambda x:len(x)>0)][:15]
imgs.media_url.apply(imageScroll.downloadImages)
print("done.")

https://pbs.twimg.com/media/FsbHoABacAAqvtt.jpg, https://pbs.twimg.com/media/FsbBitEXsAAuCxk.jpg, https://pbs.twimg.com/media/Fsa-MhBWIAAyTva.jpg, https://pbs.twimg.com/media/Fsa6LWoaQAIhSU5.jpg, https://pbs.twimg.com/media/FsatLtnXwAccK5T.jpg, https://pbs.twimg.com/media/FsaivfHaUAAkCzB.jpg, https://pbs.twimg.com/media/FsagqjuaUAAErN_.png, https://pbs.twimg.com/media/FsacRBSXwAQwBg5.jpg, https://pbs.twimg.com/media/FsaVYbPWcAMs7_X.jpg, https://pbs.twimg.com/media/FsaPCQ-WYAEbmAu.png, https://pbs.twimg.com/media/FsaIxyJWAAgcs-s.jpg, https://pbs.twimg.com/media/FsaHYhbWwAATjtH.jpg, https://pbs.twimg.com/media/FsaGHN_WAAQHnG8.jpg, https://pbs.twimg.com/media/FsaHXnyWcAASlUQ.jpg, https://pbs.twimg.com/media/FsaHYOZWABolsle.jpg, https://pbs.twimg.com/media/FsZ-7QpWAA4tyq6.jpg, done.


In [18]:
from IPython.display import Image

@interact(n=(0,50))
def scrollImages(n=0):
    
    filename=imageScroll.displayImage(imgs,n,targetFolder="IMG_Download/")
    
    return Image(filename)

interactive(children=(IntSlider(value=0, description='n', max=50), Output()), _dom_classes=('widget-interact',…

# Timerow analysis

In [19]:
@interact(column=["count","subjectivity","polarity"],normalize=False,resample=["1H","12H","1D","2D","3D","1W","2W","M","Q"])   
def TweetHist(column="count",normalize=False,resample="1H",start='2010-01-14',end='2023-05-06'):
    analysis.TweetsHist(df,name=column, resample=resample,start=start,end=end, normalize=normalize)

interactive(children=(Dropdown(description='column', options=('count', 'subjectivity', 'polarity'), value='cou…

# Timerow Analysis 2: users, hashtags, and words

In [20]:
@interact(n=(2,20),name=["username","name","hashtags","NoStopwords"],resample=["1H","12H","1D","2D","3D","1W","2W","M","Q"])
def userTimeseries(n=3,name="username",resample="D",start="2010-01-01",end="2023-30-20",stacked=True):
    analysis.userTimeseries(df,name=name,n=n,resample=resample,start=start,end=end,stacked=stacked)

interactive(children=(IntSlider(value=3, description='n', max=20, min=2), Dropdown(description='name', options…

# Social Networks: Graph Analysis

In [21]:
@interact(minDegree=(1,50),removeIsolates=True,only_largest_component=True)
def NetworkAnalysis(minDegree=30,removeIsolates=True,only_largest_component=True):
    G=analysis.NetworkGen(df)
    G=analysis.CleanGraph(G,removeIsolates=removeIsolates,minDegree=minDegree,only_largest_component=only_largest_component)
    analysis.writeNetworkHTML(G,view=True)

interactive(children=(IntSlider(value=30, description='minDegree', max=50, min=1), Checkbox(value=True, descri…